In [6]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("dataset/US_Accidents_March23.csv")

In [7]:
df = df.drop(columns=['End_Lat', 'End_Lng'], errors='ignore')

In [8]:
date_columns = ['Start_Time', 'End_Time', 'Weather_Timestamp']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [9]:
df = df.dropna(subset=['Weather_Timestamp', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction'])

In [10]:
weather_fill_columns = ['Wind_Chill(F)', 'Precipitation(in)', 'Wind_Speed(mph)']
for col in weather_fill_columns:
    df[col] = df[col].fillna(df[col].median())

In [11]:
if 'Country' in df.columns and df['Country'].nunique() == 1:
    df = df.drop(columns=['Country'])

In [12]:
df['Start_Hour'] = df['Start_Time'].dt.hour
df['Start_Weekday'] = df['Start_Time'].dt.weekday
df['Start_Month'] = df['Start_Time'].dt.month
df['Start_Year'] = df['Start_Time'].dt.year

In [13]:
df['Duration(min)'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60

In [14]:
cat_columns = ['Source', 'State', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

for col in cat_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

In [15]:
bool_columns = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 
                'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 
                'Traffic_Signal', 'Turning_Loop']

df[bool_columns] = df[bool_columns].astype(int)

In [16]:
num_columns = ['Temperature(F)', 'Wind_Speed(mph)', 'Precipitation(in)', 
               'Humidity(%)', 'Pressure(in)', 'Visibility(mi)']

for col in num_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

Handling Outliers

In [17]:
df[col] = np.clip(df[col], lower_bound, upper_bound)

In [18]:
df = df.drop_duplicates()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_') 

In [21]:
cleaned_file_path = "dataset/cleaned_us_accident_data.csv"
df.to_csv(cleaned_file_path, index=False)